In [ ]:
from pathlib import Path
import re
import subprocess
from typing import Optional
import yaml

from linkml.validator.loaders import TsvLoader
from linkml.utils.schema_builder import SchemaBuilder

from linkml_runtime.linkml_model import SlotDefinition
from linkml_runtime import SchemaView

from linkml_map.session import Session
from linkml_map.transformer.object_transformer import ObjectTransformer

In [ ]:
# Fix malformed yaml
def quote_expr_values(yaml_text):
    def replacer(match):
        indent = match.group(1)
        value = match.group(2).strip()
        
        if value.startswith('"') or value.startswith("'"):
            if "#" in value and (value.endswith('"') or value.endswith("'")):
                quote_char = value[0]
                strip_comment = value[1:-1].split('#')[0].strip()
                return f'{indent}expr: {quote_char}{strip_comment}{quote_char}'
            return match.group(0)
        
        value = value.split('#')[0].strip()
        if re.match(r'^[\w{}\s\*\+\-/().]+$', value):
            return f'{indent}expr: "{value}"'
        return match.group(0)

    pattern = re.compile(r'^(\s*)expr:\s+(.*)', re.MULTILINE)
    return pattern.sub(replacer, yaml_text)

In [ ]:
def extract_quantity_subslots(slot_derivs, doc_index, parent_slot_name, populated_from, known_subslots):
    quantity_subslots = {}

    # 1. Pull nested subslots under e.g. range_low: { value_decimal: ... }
    nested = slot_derivs.pop(parent_slot_name, {}) if isinstance(slot_derivs.get(parent_slot_name), dict) else {}

    # 2. Scan for both nested and top-level flat entries
    for subkey in known_subslots:
        # Support flat key: e.g., value_quantity.unit
        flat_key = f"{parent_slot_name}.{subkey}"

        entry = (
            nested.pop(subkey, None)
            or slot_derivs.pop(flat_key, None)
            or slot_derivs.pop(subkey, None)  # e.g., value_decimal defined at top-level
        )

        if entry:
            if isinstance(entry, dict):
                pf = entry.get("populated_from")
                expr = entry.get("expr")

                if pf and isinstance(pf, dict) and "expr" in pf:
                    quantity_subslots[subkey] = {"expr": pf["expr"]}
                elif expr:
                    quantity_subslots[subkey] = {"expr": expr}
                elif pf is None:
                    raise ValueError(f"[Doc {doc_index}] `{subkey}` in `{parent_slot_name}` has empty `populated_from:` and no `expr:`")
                elif isinstance(pf, str):
                    quantity_subslots[subkey] = {"populated_from": pf}
                else:
                    raise ValueError(f"[Doc {doc_index}] Malformed `{subkey}` in `{parent_slot_name}`: {entry}")
            else:
                raise ValueError(f"[Doc {doc_index}] Unexpected format for `{subkey}` in `{parent_slot_name}`: {entry}")

    if quantity_subslots:
        return {
            "object_derivations": [{
                "class_derivations": {
                    "Quantity": {
                        "populated_from": populated_from,
                        "slot_derivations": quantity_subslots
                    }
                }
            }]
        }

    return None

def refactor_value_quantity(documents):
    updated_docs = []

    quantity_subslots = ["value_decimal", "value_concept", "value_integer", "unit"]

    for doc_index, doc in enumerate(documents):
        cd = doc.get("class_derivations", {})
        for cls_name, cls_info in cd.items():
            slot_derivs = cls_info.get("slot_derivations", {})
            populated_from = cls_info.get("populated_from")

            # Apply to each target "container" slot
            for target_slot in ["value_quantity", "range_low", "range_high"]:
                result = extract_quantity_subslots(
                    slot_derivs,
                    doc_index,
                    target_slot,
                    populated_from,
                    quantity_subslots
                )
                if result:
                    slot_derivs[target_slot] = result

        updated_docs.append(doc)

    return updated_docs


In [ ]:
def update_populated_from_with_pht(documents, phv_to_pht):
    def find_first_phv_in_slot(slot_derivations):
        for slot_value in slot_derivations.values():
            if isinstance(slot_value, dict):
                pf = slot_value.get("populated_from")
                expr = slot_value.get("expr")

                if isinstance(pf, str) and pf.startswith("phv"):
                    return pf
                if isinstance(expr, str):
                    match = re.search(r"(phv\d{8})", expr)
                    if match:
                        return match.group(1)
        return None

    def update_class_derivations(cls_derivations, doc_index, context="root", parent_pht=None):
        for cls_name, cls_info in cls_derivations.items():
            slot_derivs = cls_info.get("slot_derivations", {})
            pf = cls_info.get("populated_from")

            if pf == "FHS":
                phv = find_first_phv_in_slot(slot_derivs)
                if phv and phv in phv_to_pht:
                    new_pf = phv_to_pht[phv]
                    cls_info["populated_from"] = new_pf
                    parent_pht = new_pf  # propagate to children
                    # print(f"✅ Updated {context}.{cls_name} populated_from: {phv} -> {new_pf}")
                elif parent_pht:
                    cls_info["populated_from"] = parent_pht
                else:
                    print(f"⚠️ Warning: No matching phv for {context}.{cls_name} in doc {doc_index}")

            # Recurse into nested object_derivations
            for slot_name, slot_value in slot_derivs.items():
                if isinstance(slot_value, dict) and "object_derivations" in slot_value:
                    for obj in slot_value["object_derivations"]:
                        inner_cls_derivs = obj.get("class_derivations")
                        if inner_cls_derivs:
                            update_class_derivations(inner_cls_derivs, doc_index, context=f"{context}.{cls_name}.{slot_name}", parent_pht=parent_pht)

    for doc_index, doc in enumerate(documents):
        top_cd = doc.get("class_derivations", {})
        update_class_derivations(top_cd, doc_index)

    return documents

def load_phv_to_pht_map(file_path):
    with open(file_path) as f:
        return dict(line.strip().split(": ") for line in f if line.strip())


In [ ]:
def fix_misindented_value_mappings(yaml_text):
    lines = yaml_text.splitlines()
    fixed = []
    i = 0
    while i < len(lines):
        line = lines[i]
        fixed.append(line)
        if re.match(r'^\s+populated_from:\s+\S+', line):
            pf_indent = len(line) - len(line.lstrip())
            # Peek ahead to see if value_mappings is wrongly indented
            if i + 1 < len(lines) and lines[i + 1].lstrip().startswith("value_mappings:"):
                vm_line = lines[i + 1]
                vm_indent = len(vm_line) - len(vm_line.lstrip())
                if vm_indent > pf_indent:
                    # Dedent value_mappings
                    fixed.append(" " * pf_indent + vm_line.lstrip())
                    i += 1
                    # Dedent any following lines more indented than vm_indent
                    while i + 1 < len(lines):
                        peek = lines[i + 1]
                        peek_indent = len(peek) - len(peek.lstrip())
                        if peek_indent > vm_indent:
                            i += 1
                            fixed.append(" " * (pf_indent + 2) + peek.lstrip())
                        else:
                            break
        i += 1
    return "\n".join(fixed)


In [ ]:
def fix_str_fields_expr(parsed_docs):
    for doc in parsed_docs:
        for cls in doc.get("class_derivations", {}).values():
            slot_derivs = cls.get("slot_derivations", {})
            for slot_name, slot_value in list(slot_derivs.items()):
                if isinstance(slot_value, str):
                    slot_derivs[slot_name] = {"expr": f"'{slot_value}'"}
    return parsed_docs


In [ ]:
def quote_expr_strings(obj):
    if isinstance(obj, dict):
        new_obj = {}
        for k, v in obj.items():
            if k == "expr" and isinstance(v, str):
                # Only add quotes if not already quoted
                if not (v.startswith("'") and v.endswith("'")):
                    v = f"'{v}'"
            new_obj[k] = quote_expr_strings(v)
        return new_obj
    elif isinstance(obj, list):
        return [quote_expr_strings(i) for i in obj]
    else:
        return obj


In [ ]:
EXPR_PATTERN = re.compile(r'\b(case)\b|[()+*/<>]|(?<!\w)-(?=\w)')
def is_const_expr(expr: str) -> bool:
    expr = expr.strip().strip("'\"")
    if not expr:
        return False
    # If the expression contains any expression-like elements, it's not constant
    if EXPR_PATTERN.search(expr):
        return False
    return True

def const_expr_to_value_in_slot(slot_deriv: dict) -> None:
    if not isinstance(slot_deriv, dict):
        return
    expr = slot_deriv.get("expr")
    if isinstance(expr, str) and is_const_expr(expr):
        slot_deriv["value"] = expr.strip("'\"")
        slot_deriv.pop("expr", None)
        slot_deriv.setdefault("range", "string")

def const_expr_to_value_in_class(class_deriv: dict) -> None:
    slot_derivs = class_deriv.get("slot_derivations", {})
    for slot_deriv in slot_derivs.values():
        const_expr_to_value_in_slot(slot_deriv)

        # Recurse into object_derivations
        if isinstance(slot_deriv, dict):
            for obj in slot_deriv.get("object_derivations", []):
                for obj in slot_deriv.get("object_derivations", []):
                    if isinstance(obj, dict) and "slot_derivations" in obj:
                        const_expr_to_value_in_class(obj)


def const_expr_to_value(parsed_docs: list[dict]) -> list[dict]:
    for doc in parsed_docs:
        for class_deriv in doc.get("class_derivations", {}).values():
            const_expr_to_value_in_class(class_deriv)
    return parsed_docs


In [ ]:
def extract_inline_comments(parsed_docs: list[dict]) -> list[dict]:
    def extract_comment_from_value(val: str) -> tuple[str, Optional[str]]:
        if "#" in val:
            main, comment = val.split("#", 1)
            return main.strip(), comment.strip()
        return val, None

    def fix_slot(slot: dict):
        if not isinstance(slot, dict):
            return
        for field in ("value", "expr"):
            val = slot.get(field)
            if isinstance(val, str) and "#" in val:
                clean_val, comment = extract_comment_from_value(val)
                slot[field] = clean_val
                if comment:
                    slot.setdefault("comments", []).append(comment)

    def recurse_class(class_deriv: dict):
        for slot_deriv in class_deriv.get("slot_derivations", {}).values():
            fix_slot(slot_deriv)
            for obj in slot_deriv.get("object_derivations", []):
                if isinstance(obj, dict):
                    recurse_class(obj)

    for doc in parsed_docs:
        for class_deriv in doc.get("class_derivations", {}).values():
            recurse_class(class_deriv)

    return parsed_docs


In [ ]:
def fix_nested_expr_under_populated_from(yaml_text: str) -> str:
    lines = yaml_text.splitlines()
    out_lines = []
    i = 0

    while i < len(lines):
        line = lines[i]

        if line.lstrip().startswith("populated_from:"):
            indent = len(line) - len(line.lstrip())
            next_line = lines[i + 1] if i + 1 < len(lines) else ""

            if next_line.lstrip().startswith("expr:"):
                # Rewrite: replace populated_from + expr with dedented expr
                out_lines.append(" " * indent + next_line.lstrip())
                i += 2

                # Append any deeper-indented lines
                while i < len(lines):
                    peek = lines[i]
                    if len(peek) - len(peek.lstrip()) > indent:
                        out_lines.append(" " * (indent + 2) + peek.lstrip())
                        i += 1
                    else:
                        break
                continue

        out_lines.append(line)
        i += 1

    return "\n".join(out_lines)


In [ ]:
def strip_braces_in_exprs(yaml_text: str) -> str:
    lines = yaml_text.splitlines()
    new_lines = []
    in_expr_block = False

    for line in lines:
        stripped = line.lstrip()
        if stripped.startswith("expr:"):
            # Replace braces on the line
            line = re.sub(r"\{([^}]+)\}", r"\1", line)
            in_expr_block = True
        elif in_expr_block:
            indent = len(line) - len(stripped)
            if indent > 0:
                # Still part of the expression block
                line = re.sub(r"\{([^}]+)\}", r"\1", line)
            else:
                in_expr_block = False
        new_lines.append(line)

    return "\n".join(new_lines)


In [ ]:
def quote_unquoted_exprs(yaml_text: str) -> str:
    lines = yaml_text.splitlines()
    out_lines = []
    i = 0

    while i < len(lines):
        line = lines[i]
        stripped = line.lstrip()
        indent = len(line) - len(stripped)

        if stripped.startswith("expr:"):
            expr_val = stripped[len("expr:"):].lstrip()

            # Skip already quoted expressions
            if expr_val.startswith(("'", '"')):
                out_lines.append(line)
                i += 1
                continue

            # Start quoting
            quoted_line = " " * indent + "expr: \""
            remainder = expr_val
            i += 1

            # Collect multiline expr block (indented more than current line)
            while i < len(lines):
                next_line = lines[i]
                next_indent = len(next_line) - len(next_line.lstrip())
                if next_indent > indent:
                    remainder += "\\n" + next_line.strip()
                    i += 1
                else:
                    break

            # ✅ FIX: Ensure phv000#### references are wrapped in braces
            remainder = re.sub(r'\b(phv\d{8})\b', r'{\1}', remainder)

            # ✅ Quote and escape
            quoted_line += remainder.replace('"', '\\"') + "\""
            out_lines.append(quoted_line)
        else:
            out_lines.append(line)
            i += 1

    return "\n".join(out_lines)


In [ ]:
spec_dir = Path("/sbgenomics/workspace/NHLBI-BDC-DMC-HV/priority_variables_transform/FHS")
data_dir = "/sbgenomics/workspace/output/FHS_v31_c1"
output_dir = Path("/sbgenomics/workspace/NHLBI-BDC-DMC-HV/priority_variables_transform/FHS-ingest")

# Not complete: bld_pressure
# unreasolvable issues: ven_thromb, valv_hrtdis, whtbld_ct
# SKIP_BASES = {"afib", # "fast_gluc_bld", "bld_pressure",
#               # "glucose_bld", "hdl", "ldl", "hip_circ",
#               # "alt_sgpt", "bdy_hgt", "bdy_wgt",  "cig_smok", "creat_bld", "edu_lvl",
#               # "fam_income", "hist_cor_angio", "hist_cor_bypg", "hist_my_inf", "hrtrt", "hypertension", "nt_bnp", "stroke",
#               # "tak_betablk", "tak_calchanblk", "tak_diuret", "tot_chol_bld", "triglyc_bld", "troponin", "waist_circ",
#              }
SKIP_BASES = {"bld_pressure"}
RUN_BASES = {"demography"}

for yaml_file in spec_dir.glob("*.yaml"):
    base = yaml_file.stem  # Strip .yaml
    output_file = f"{output_dir}/{base}.yaml"

    # if base in SKIP_BASES:
    #     # print(f"⏭️ Skipping {base}")
    #     continue

    if base not in RUN_BASES:
    # print(f"⏭️ Skipping {base}")
        continue

    # if Path(output_file).exists():
    #     print(f"⏭️ Output exists, skipping {base}")
    #     continue
    
    print(base)

    ### Load and fix issues then process YAML
    raw = "\n".join(line.rstrip() for line in yaml_file.read_text().splitlines())
    # Fix trailing spaces
    pre_yaml = "\n".join(line.rstrip() for line in raw.splitlines())

    # Fix when there is not a space after a colon
    pre_yaml = re.sub(r'^(\s*[^#:\n]+?):(?=\S)', r'\1: ', pre_yaml, flags=re.MULTILINE)
    # quoted_fixed = quote_expr_values(colon_fixed)

    # Fix improper indenting on value_mappings
    pre_yaml = fix_misindented_value_mappings(pre_yaml)

    # Fix improper nesting of expr under populated_from
    pre_yaml = fix_nested_expr_under_populated_from(pre_yaml)

    # Add quoting around expressions, including multiline
    pre_yaml = quote_unquoted_exprs(pre_yaml)

    # # Fix improper nesting of expr under populated_from
    # pre_yaml = strip_braces_in_exprs(pre_yaml)
    
    # Fix UNKNOWN participant in certain files.
    pre_yaml = re.sub(
        r'(associated_participant:\n\s+)(expr:\s+\'\'?UNKNOWN\'\'?)',
        r'\1populated_from: phv00277066',
        pre_yaml
    )

    # Split blocks class derivations into proper YAML blocks
    pre_yaml = re.split(r'(?<=\n)(?=^\s*class_derivations:\s*)', pre_yaml, flags=re.MULTILINE)

    # Parse the yaml
    parsed_yaml = [yaml.safe_load(doc) for doc in pre_yaml]

    # convert const expressions to value entity
    parsed_yaml = const_expr_to_value(parsed_yaml)

    # turn quantity 
    parsed_yaml = refactor_value_quantity(parsed_yaml)

    # Run the shell command to regenerate phv dict
    result = subprocess.run(
        f"""for phv in $(grep -ho 'phv[0-9]\\{{8\\}}' {spec_dir}/{base}.yaml | sort -u); do \
        grep -l "$phv" {data_dir}/*.tsv | \
        sed -E "s|.*/(pht[0-9]{{6,}}).tsv|$phv: \\1|"; done""",
        shell=True, check=True, capture_output=True, text=True,
    )
    phv_to_pht = dict(line.split(": ") for line in result.stdout.strip().splitlines())

    parsed_yaml = update_populated_from_with_pht(parsed_yaml, phv_to_pht)

    parsed_yaml = extract_inline_comments(parsed_yaml)

    # Quote Expr strings
    # parsed_yaml = quote_expr_strings(parsed_yaml)

    # Dump to YAML
    with open(output_file, "w") as f:
        yaml.dump(parsed_yaml, f, sort_keys=False, allow_unicode=True)
    # break

In [ ]:
print(pre_yaml[1])
# print(yaml.dump(pht_replace_docs))

In [ ]:
# Run the shell command to regenerate phv_to_pht.txt
base = "bdy_wgt"
subprocess.run(
    f"""for phv in $(grep -ho 'phv[0-9]\\{{8\\}}' {spec_dir}/{base}.yaml | sort -u); do \
    grep -l "$phv" {data_dir}/*.tsv | \
    sed -E "s|.*/(pht[0-9]{{6,}}).tsv|$phv: \\1|"; done > phv_to_pht.txt""",
    shell=True, check=True,
)

In [ ]:
# Set base name
spec_dir = "NHLBI-BDC-DMC-HV/priority_variables_transform/FHS"
base = "afib"
data_dir = "output/FHS_v31_c1"

# Run the shell command to regenerate phv_to_pht.txt
subprocess.run(
    f"""for phv in $(grep -ho 'phv[0-9]\\{{8\\}}' {spec_dir}/{base}.yaml | sort -u); do \
    grep -l "$phv" {data_dir}/*.tsv | \
    sed -E "s|.*/(pht[0-9]{{6,}}).tsv|$phv: \\1|"; done > phv_to_pht.txt""",
    shell=True, check=True,
)

# Load and process the YAML
raw = Path(f"NHLBI-BDC-DMC-HV/priority_variables_transform/FHS/{base}.yaml").read_text()
quoted_fixed = quote_expr_values(raw)
split_blocks = re.split(r'(?<=\n)(?=^\s*class_derivations:\s*)', quoted_fixed, flags=re.MULTILINE)
parsed_docs = [yaml.safe_load(doc) for doc in split_blocks]

refactored_docs = refactor_value_quantity(parsed_docs)

phv_to_pht = load_phv_to_pht_map("NHLBI-BDC-DMC-HV/priority_variables_transform/FHS/phv_to_pht.txt")

pht_replace_docs = update_populated_from_with_pht(refactored_docs, phv_to_pht)

# Dump to YAML
with open(f"NHLBI-BDC-DMC-HV/priority_variables_transform/FHS-ingest/{base}.yaml", "w") as f:
    yaml.dump(pht_replace_docs, f, sort_keys=False, allow_unicode=True)


In [ ]:
# Person - top level class
person_yaml = yaml.safe_load("""
class_derivations:
  Person:
    populated_from: pht000009
    slot_derivations:
      species:
        expr: "'Homo Sapiens'"
      identity:
        populated_from: dbGaP_Subject_ID
""")

# # Dump to YAML
# with open(var_dir + "person" + ".yaml", "w") as f:
#     yaml.dump(person_yaml, f, sort_keys=False, allow_unicode=True)

In [ ]:
source_sv = SchemaView("/sbgenomics/workspace/output/Schema_FHS_v31_c1/schema-automator-data/Schema_FHS_v31_c1.yaml")
source_schema = source_sv.schema

target_sv = SchemaView("/sbgenomics/workspace/NHLBI-BDC-DMC-HM/src/bdchm/schema/bdchm.yaml")
target_schema = target_sv.schema

var_dir = "/sbgenomics/workspace/NHLBI-BDC-DMC-HV/priority_variables_transform/FHS-ingest/"

In [ ]:
transform_spec = person_yaml

data_loader = TsvLoader("/sbgenomics/workspace/output/FHS_v31_c1/pht000009.tsv")
data_rows = data_loader.iter_instances()

first_row = next(data_rows)
cur_row = first_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(cur_row, source_type="pht000009")

print(result)
print("Transformation Successful!")

In [ ]:
# Participant - top level class for study data
participant_yaml = yaml.safe_load("""
class_derivations:
  Participant:
    populated_from: pht000395
    slot_derivations:
      # associated_participant: 
      #   populated_from: phv00007675
      identity:
        populated_from: dbGaP_Subject_ID
      member_of_research_study:
        expr: "'FHS'"
""")

# # Dump to YAML
# with open(var_dir + "participant" + ".yaml", "w") as f:
#     yaml.dump(participant_yaml, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Load MeasurementObservation class derivations
bdy_hgt = refactored_docs
# bdy_hgt = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bdy_hgt" + ".yaml")))
# bdy_wgt = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bdy_wgt" + ".yaml")))
# bmi = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bmi" + ".yaml")))
# bp_diastolic = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bp_diastolic" + ".yaml")))
# bp_systolic = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bp_systolic" + ".yaml")))
# fev1 = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "fev1" + ".yaml")))
# fev1_fvc = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "fev1_fvc" + ".yaml")))
# fvc = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "fvc" + ".yaml")))
# hrt_rt = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "hrt_rt" + ".yaml")))
# spo2 = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "spo2" + ".yaml")))

# Get the demography slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_exposures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("exposures", {})

# Add the Demography object_derivation to the demography slot
participant_exposures_slot.setdefault("object_derivations", bdy_hgt)
# participant_exposures_slot.setdefault("object_derivations", [
#     bdy_hgt,
#     tak_betablk,
#     tak_adrenergics,
#     tak_cort_steroid_resp,
#     tak_cort_steroid_oral,
#     tak_anabolic_steroid,
# ])

In [ ]:
transform_spec = participant_yaml

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="pht000009")

print(result)
print("Transformation Successful!")

In [ ]:
# Let's start with one simple condition
angina = yaml.safe_load(open(str(var_dir + "angina" + ".yaml")))
# print(yaml.dump(angina))

# Get the conditions slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_exposures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("conditions", {})

# Add the conditions object_derivation to the demography slot
# participant_exposures_slot.setdefault("object_derivations", [
#     angina,
#     # asthma,
#     # copd,
#     # diabetes,
#     # hist_hrt_failure,
#     # hist_my_inf,
#     # hyperten,
#     # pad,
#     # slp_ap,
#     # stroke,
#     # stroke_isch_atk,
# ])

participant_exposures_slot.setdefault("object_derivations", angina)

In [ ]:
data_loader = TsvLoader("/sbgenomics/workspace/output/FHS_v31_c1/pht000395.tsv")
data_rows = data_loader.iter_instances()

first_row = next(data_rows)
cur_row = first_row

transform_spec = participant_yaml

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="pht000395")

print(result)
print("Transformation Successful!")

In [ ]:
transform_spec = refactored_docs


data_loader = TsvLoader("/sbgenomics/workspace/output/FHS_v31_c1/pht000009.tsv")
data_rows = data_loader.iter_instances()

first_row = next(data_rows)
cur_row = first_row

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="pht000009")

print(result)
print("Transformation Successful!")

In [ ]:
# Turn sequential class_derivations (malformed yaml) into list of class derivations.

# Function to fix unquoted expr: text (malformed yaml)
def quote_expr_values(yaml_text):
    def replacer(match):
        indent = match.group(1)
        value = match.group(2).strip()

        # Don't quote if already quoted OR looks like a quoted literal
        if value.startswith('"') or value.startswith("'"):
            return match.group(0)

        # Don't quote if it's a simple scalar (e.g., a single variable)
        if re.match(r'^[\w{}\s\*\+\-/().]+$', value):
            return f'{indent}expr: "{value}"'

        # Otherwise, leave it as-is
        return match.group(0)

    pattern = re.compile(r'^(\s*)expr:\s+(.*)', re.MULTILINE)
    return pattern.sub(replacer, yaml_text)

# Read the raw YAML as text
raw = Path("NHLBI-BDC-DMC-HV/priority_variables_transform/FHS/bdy_hgt_test.yaml").read_text()
# print(raw)

quoted_fixed = quote_expr_values(raw)
# Split while *keeping* 'class_derivations:' in each result, skip first line
split_blocks = re.split(r'(?<=\n)(?=^\s*class_derivations:\s*)', quoted_fixed, flags=re.MULTILINE)
# print(split_blocks)

# print(split_blocks)
parsed_docs = [yaml.safe_load(doc) for doc in split_blocks]
print(yaml.dump(parsed_docs))


raw2 = Path("NHLBI-BDC-DMC-HV/priority_variables_transform/FHS/bdy_hgt_test2.yaml").read_text()
# print(raw2)

# Split while *keeping* 'class_derivations:' in each result, skip first line
split_blocks2 = re.split(r'(?<=\n)(?=^\s*class_derivations:\s*)', raw2, flags=re.MULTILINE)
# print(split_blocks2)
print(split_blocks == split_blocks2)
# quoted_fixed2 = quote_expr_values(raw2)
# print(quoted_fixed2)
parsed_docs2 = [yaml.safe_load(doc) for doc in split_blocks2]
# print(yaml.dump(parsed_docs2))

# parsed_docs = [yaml.safe_load(doc) for doc in quoted_fixed]

# Use these if we can't guarantee the first class_derivation is first line of file.
# # Split while *keeping* 'class_derivations:' in each result
# split_blocks = re.split(r'(?=^\s*class_derivations:\s*)', raw, flags=re.MULTILINE)
# # First block is likely empty or comments/header — skip it
# blocks = [b for b in split_blocks[1:]]
# parsed_docs = [yaml.safe_load(doc) for doc in blocks]

# print(yaml.dump(quoted_fixed))

In [ ]:
# Turn sequential class_derivations (malformed yaml) into list of class derivations.

# Read the raw YAML as text
raw = Path("NHLBI-BDC-DMC-HV/priority_variables_transform/FHS/apnea_hypop_index.yaml").read_text()
print(raw)

# Split while *keeping* 'class_derivations:' in each result, skip first line
split_blocks = re.split(r'(?<=\n)(?=^\s*class_derivations:\s*)', raw, flags=re.MULTILINE)
parsed_docs = [yaml.safe_load(doc) for doc in split_blocks]

# Use these if we can't guarantee the first class_derivation is first line of file.
# # Split while *keeping* 'class_derivations:' in each result
# split_blocks = re.split(r'(?=^\s*class_derivations:\s*)', raw, flags=re.MULTILINE)
# # First block is likely empty or comments/header — skip it
# blocks = [b for b in split_blocks[1:]]
# parsed_docs = [yaml.safe_load(doc) for doc in blocks]

print(yaml.dump(parsed_docs))

In [ ]:
import csv
from pathlib import Path

class LazySubjectDict(dict):
    """
    Lazily loads per-pht data for a single subject on demand.
    """
    def __init__(self, subject_id, data_dir):
        super().__init__()
        self.subject_id = subject_id
        self.data_dir = Path(data_dir)
        self._cache = {}

    def __getitem__(self, pht_id):
        if pht_id in self._cache:
            return self._cache[pht_id]

        file_path = self.data_dir / f"{pht_id}.tsv"
        if not file_path.exists():
            raise KeyError(f"No such file: {file_path}")

        with open(file_path, newline='') as f:
            reader = csv.DictReader(f, delimiter="\t")
            for row in reader:
                if row.get("dbGaP_Subject_ID") == self.subject_id:
                    self._cache[pht_id] = row
                    return row

        raise KeyError(f"Subject {self.subject_id} not found in {pht_id}")

In [ ]:
source_sv = SchemaView("/sbgenomics/workspace/output/Schema_FHS_v31_c1/schema-automator-data/Schema_FHS_v31_c1.yaml")
source_schema = source_sv.schema

target_sv = SchemaView("NHLBI-BDC-DMC-HM/src/bdchm/schema/bdchm.yaml")
target_schema = target_sv.schema

data_loader = TsvLoader("/sbgenomics/workspace/output/FHS_v31_c1/pht000030.tsv")
data_rows = data_loader.iter_instances()

first_row = next(data_rows)
cur_row = first_row

var_dir = "NHLBI-BDC-DMC-HV/priority_variables_transform/FHS/"
print(cur_row)

In [ ]:
# Person - top level class
person_yaml = yaml.safe_load("""
class_derivations:
  Person:
    populated_from: pht000030
    slot_derivations:
      species:
        expr: "'Homo Sapiens'"
      identity:
        populated_from: dbGaP_Subject_ID
""")

# # Dump to YAML
# with open(var_dir + "person" + ".yaml", "w") as f:
#     yaml.dump(person_yaml, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Participant - top level class for study data
participant_yaml = yaml.safe_load("""
class_derivations:
  Participant:
    populated_from: pht000030
    slot_derivations:
      # associated_participant: 
      #   populated_from: phv00007675
      identity:
        populated_from: dbGaP_Subject_ID
      member_of_research_study:
        expr: "'FHS'"
""")

# # Dump to YAML
# with open(var_dir + "participant" + ".yaml", "w") as f:
#     yaml.dump(participant_yaml, f, sort_keys=False, allow_unicode=True)

In [ ]:
transform_spec = participant_yaml

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="pht000030")

print(result)
print("Transformation Successful!")

In [ ]:
# Get the participants slot
person_class = person_yaml.setdefault("class_derivations", {}).setdefault("Person", {})
person_participants_slot = person_class.setdefault("slot_derivations", {}).setdefault("participants", {})

# Add the Participant object_derivation to the participants slot
person_participants_slot.setdefault("object_derivations", [ participant_yaml ])

# print(yaml.dump(person_yaml, sort_keys=False))

In [ ]:
transform_spec = person_yaml

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="pht000030")

print(result)
print("Transformation Successful!")

In [ ]:
# Let's start with one simple condition
angina = yaml.safe_load(open(str(var_dir + "angina" + ".yaml")))
# print(yaml.dump(angina))

# Get the conditions slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_exposures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("conditions", {})

# Add the conditions object_derivation to the demography slot
participant_exposures_slot.setdefault("object_derivations", [
    angina,
    # asthma,
    # copd,
    # diabetes,
    # hist_hrt_failure,
    # hist_my_inf,
    # hyperten,
    # pad,
    # slp_ap,
    # stroke,
    # stroke_isch_atk,
])

In [ ]:
print(yaml.dump(person_yaml))

In [ ]:
transform_spec = person_yaml

data_loader = TsvLoader("/sbgenomics/workspace/output/FHS_v31_c1/pht000030.tsv")
data_rows = data_loader.iter_instances()

first_row = next(data_rows)
cur_row = first_row

other_data_loader = TsvLoader("/sbgenomics/workspace/output/FHS_v31_c1/pht000395.tsv")
other_data_rows = other_data_loader.iter_instances()

other_first_row = next(other_data_rows)
other_cur_row = other_first_row

input_data = {
    "pht000030": cur_row,
    "pht000395": other_cur_row
}


# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="FHS")

print(result)
print("Transformation Successful!")

In [ ]:
# Load Condition class derivations
angina = yaml.safe_load(open(str(var_dir + "condition/" + "angina" + ".yaml")))
asthma = yaml.safe_load(open(str(var_dir + "condition/" + "asthma" + ".yaml")))
copd = yaml.safe_load(open(str(var_dir + "condition/" + "copd" + ".yaml")))
diabetes = yaml.safe_load(open(str(var_dir + "condition/" + "diabetes" + ".yaml")))
hist_hrt_failure = yaml.safe_load(open(str(var_dir + "condition/" + "hist_hrt_failure" + ".yaml")))
hist_my_inf = yaml.safe_load(open(str(var_dir + "condition/" + "hist_my_inf" + ".yaml")))
hyperten = yaml.safe_load(open(str(var_dir + "condition/" + "hyperten" + ".yaml")))
pad = yaml.safe_load(open(str(var_dir + "condition/" + "pad" + ".yaml")))
slp_ap = yaml.safe_load(open(str(var_dir + "condition/" + "slp_ap" + ".yaml")))
stroke = yaml.safe_load(open(str(var_dir + "condition/" + "stroke" + ".yaml")))
stroke_isch_atk = yaml.safe_load(open(str(var_dir + "condition/" + "stroke_isch_atk" + ".yaml")))

# Get the conditions slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_exposures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("conditions", {})

# Add the conditions object_derivation to the demography slot
participant_exposures_slot.setdefault("object_derivations", [
    angina,
    asthma,
    copd,
    diabetes,
    hist_hrt_failure,
    hist_my_inf,
    hyperten,
    pad,
    slp_ap,
    stroke,
    stroke_isch_atk,
])

In [ ]:
demography_yaml = yaml.safe_load("""
class_derivations:
  Demography:
    populated_from: COPDGene
    slot_derivations:
      associated_participant:
        populated_from: phv00159568
      sex:
        populated_from: phv00159571
        value_mappings:
          '1': OMOP:8507  # MALE
          '2': OMOP:8532  # FEMALE
      ethnicity:
        populated_from: phv00159573
        value_mappings:
          '1': HISPANIC_OR_LATINO
          '2': NOT_HISPANIC_OR_LATINO
      race:
        populated_from: phv00159572
        value_mappings:
          '1': OMOP:8527
          '2': OMOP:8516
          '3': OMOP:8515
          '4': OMOP:8557
          '5': OMOP:8657
          '6': OMOP:45880900
          '7': OMOP:8552
""")

# Dump to YAML
with open(var_dir + "demography" + ".yaml", "w") as f:
    yaml.dump(demography_yaml, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Get the demography slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_demography_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("demography", {})

# Add the Demography object_derivation to the demography slot
participant_demography_slot.setdefault("object_derivations", [ demography_yaml ])

In [ ]:
# Load DrugExposure class derivations
tak_betablk_resp = yaml.safe_load(open(str(var_dir + "exposure/" + "tak_betablk_resp" + ".yaml")))
tak_betablk = yaml.safe_load(open(str(var_dir + "exposure/" + "tak_betablk" + ".yaml")))
tak_adrenergics = yaml.safe_load(open(str(var_dir + "exposure/" + "tak_adrenergics" + ".yaml")))
tak_cort_steroid_resp = yaml.safe_load(open(str(var_dir + "exposure/" + "tak_cort_steroid_resp" + ".yaml")))
tak_cort_steroid_oral = yaml.safe_load(open(str(var_dir + "exposure/" + "tak_cort_steroid_oral" + ".yaml")))
tak_anabolic_steroid = yaml.safe_load(open(str(var_dir + "exposure/" + "tak_anabolic_steroid" + ".yaml")))

# Get the demography slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_exposures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("exposures", {})

# Add the Demography object_derivation to the demography slot
participant_exposures_slot.setdefault("object_derivations", [
    tak_betablk_resp,
    tak_betablk,
    tak_adrenergics,
    tak_cort_steroid_resp,
    tak_cort_steroid_oral,
    tak_anabolic_steroid,
])

In [ ]:
# Load MeasurementObservation class derivations
bdy_hgt = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bdy_hgt" + ".yaml")))
bdy_wgt = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bdy_wgt" + ".yaml")))
bmi = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bmi" + ".yaml")))
bp_diastolic = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bp_diastolic" + ".yaml")))
bp_systolic = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "bp_systolic" + ".yaml")))
fev1 = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "fev1" + ".yaml")))
fev1_fvc = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "fev1_fvc" + ".yaml")))
fvc = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "fvc" + ".yaml")))
hrt_rt = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "hrt_rt" + ".yaml")))
spo2 = yaml.safe_load(open(str(var_dir + "measurement_observation/" + "spo2" + ".yaml")))

# Get the demography slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_exposures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("exposures", {})

# Add the Demography object_derivation to the demography slot
participant_exposures_slot.setdefault("object_derivations", [
    bdy_hgt,
    tak_betablk,
    tak_adrenergics,
    tak_cort_steroid_resp,
    tak_cort_steroid_oral,
    tak_anabolic_steroid,
])

In [ ]:
observation_yaml = yaml.safe_load("""
class_derivations:
  Observation:
    populated_from: COPDGene
    slot_derivations:
      associated_participant:
        populated_from: phv00159568
      observation_type:
        expr: "'OMOP:4282779'"  # Cigarette smoking status
      value_enum:
        expr: "'OMOP:40766945' if {phv00159749} == 1 else 'OMOP:45883458' if {phv00159747} == 1 else 'OMOP:45883537'"
""")

# Dump to YAML
with open(var_dir + "observation" + ".yaml", "w") as f:
    yaml.dump(observation_yaml, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Get the observations slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_observations_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("observations", {})

# Add the Demography object_derivation to the demography slot
participant_observations_slot.setdefault("object_derivations", [ observation_yaml ])

# print(yaml.dump(person_yaml))

In [ ]:
# hist_cor_angio
hist_cor_angio = yaml.safe_load("""
class_derivations:
  Procedure:
    populated_from: COPDGene
    slot_derivations:
      associated_participant:
        populated_from: phv00159568
      procedure_concept:
        expr: "'OMOP:4184832'"  # Coronary angioplasty
      procedure_status:
        populated_from: phv00159632
        value_mappings:
          '0': ABSENT
          '1': PRESENT
""")

# Dump to YAML
with open(var_dir + "procedure/" + "hist_cor_angio" + ".yaml", "w") as f:
    yaml.dump(hist_cor_angio, f, sort_keys=False, allow_unicode=True)

In [ ]:
transform_spec = hist_cor_angio

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

result = transformer.map_object(input_data, source_type="COPDGene")

print(result)
print("Transformation Successful!")

In [ ]:
# hist_cor_bypg
hist_cor_bypg = yaml.safe_load("""
class_derivations:
  Procedure:
    populated_from: COPDGene
    slot_derivations:
      associated_participant:
        populated_from: phv00159568
      procedure_concept:
        expr: "'OMOP:4336464'"  #coronary bypass graft
      procedure_status:
        populated_from: phv00159631
        value_mappings:
          '0': ABSENT
          '1': PRESENT
""")

# Dump to YAML
with open(var_dir + "procedure/" + "hist_cor_bypg" + ".yaml", "w") as f:
    yaml.dump(hist_cor_bypg, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Get the observations slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_procedures_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("procedures", {})

# Add the Demography object_derivation to the demography slot
participant_procedures_slot.setdefault("object_derivations", [ hist_cor_angio, hist_cor_bypg ])

In [ ]:
# edu_lvl
edu_lvl = yaml.safe_load("""
class_derivations:
  SdohObservation:
    populated_from: COPDGene
    slot_derivations:
      associated_participant:
        populated_from: phv00159568
      category:
        expr: "'EDUCATIONAL_ATTAINMENT'"
      value_enum:
        populated_from: phv00159773
        value_mappings:
          '1': 8TH_GRADE_OR_LESS
          '2': HIGH_SCHOOL_NO_DIPLOMA
          '3': HIGH_SCHOOL_GRADUATE_GED
          '4': SOME_COLLEGE_OR_TECH_NO_DEGREE
          '5': COLLEGE_OR_TECH_WITH_DEGREE
          '6': MASTERS_OR_DOCTORAL_DEGREE
""")

# Dump to YAML
with open(var_dir + "sdoh_observation/" + "edu_lvl" + ".yaml", "w") as f:
    yaml.dump(edu_lvl, f, sort_keys=False, allow_unicode=True)

In [ ]:
# Get the observations slot on Participants class
participant_cls = participant_yaml.setdefault("class_derivations", {}).setdefault("Participant", {})
participant_sdoh_observations_slot = participant_cls.setdefault("slot_derivations", {}).setdefault("sdoh_observations", {})

# Add the Demography object_derivation to the demography slot
participant_sdoh_observations_slot.setdefault("object_derivations", [ edu_lvl ])

In [ ]:
transform_spec = person_yaml

input_data = cur_row

# Create ObjectTransformer and apply transformation
transformer = ObjectTransformer(unrestricted_eval=True)
transformer.source_schemaview = SchemaView(source_schema)
transformer.target_schemaview = SchemaView(target_schema)
transformer.create_transformer_specification(transform_spec)

# Transform all rows
output_data = []
for row in data_rows:
    result = transformer.map_object(row, source_type="COPDGene")
    if result:  # Avoid None or empty dicts
        output_data.append(result)

# Final wrapped structure (key should match the collection slot, or be schema-compatible)
wrapped_output = {
    "persons": output_data
}

# Dump to YAML
with open("transformed_person_data_DS_CS.yaml", "w") as f:
    yaml.dump(wrapped_output, f, sort_keys=False, allow_unicode=True)

print("Transformation Successful!")

In [ ]:
# Dump final Person class to YAML
with open(var_dir + "person" + ".yaml", "w") as f:
    yaml.dump(person_yaml, f, sort_keys=False, allow_unicode=True)